<div style="text-align: center; font-size: 30px; color: blue; font-family: 'Arial', sans-serif;">
    <b>"Clasificación automática de reseñas negativas"</b>
</div>

# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Carga y Exploración de los Datos](#data_review)
    * [1.1 Descripción de los Datos](#data_review)
* [Etapa 2. Preparación de los Datos](#data_preparing)
* [Etapa 3. Entrenamiento de modelos](#data_training)
* [Conclusiones finales del proyecto](#end)

# Etapa 1. Carga y Exploración de los Datos <a id=data_review></a>

In [14]:
# Cargamos los módulos a utilizar, e importamos funciones específicas de módulos
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Albert\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Albert\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Albert\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Introducción

En la actualidad, las reseñas de películas se han convertido en una fuente clave de información para los espectadores que desean tomar decisiones informadas sobre qué películas ver. Sin embargo, con la gran cantidad de reseñas disponibles en plataformas como IMDB, el proceso de análisis de estas opiniones se vuelve abrumador. La automatización de este proceso es esencial para filtrar y clasificar eficientemente las reseñas en categorías como positivas y negativas.

Este proyecto tiene como objetivo desarrollar un sistema que permita la clasificación automática de reseñas de películas, enfocándose en la detección de críticas negativas. Utilizando un conjunto de datos de reseñas de películas etiquetadas de IMDB, se entrenarán y evaluarán diversos modelos de machine learning para identificar las críticas negativas con una precisión y eficacia superiores. A lo largo del proyecto, se realizará un análisis exhaustivo de los datos, se preprocesarán las reseñas, se entrenarán diferentes modelos de clasificación, y se evaluarán sus desempeños utilizando métricas estándar como el valor F1, con el objetivo de alcanzar un umbral mínimo de 0.85.

Este sistema contribuirá a mejorar la experiencia del usuario en la comunidad Film Junky Union, permitiendo filtrar rápidamente reseñas negativas y ayudando a los aficionados al cine a tomar decisiones más informadas y eficientes sobre las películas que eligen ver.


In [2]:
# Importamos el DF
df_review = pd.read_csv('/home/josue/Clasificación_automática_de_reseñas_negativas/imdb_reviews.tsv', sep='\t')

# 1.1 Descripción de los Datos <a id='data_reviw'></a>

In [3]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47331 entries, 0 to 47330
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           47331 non-null  object 
 1   title_type       47331 non-null  object 
 2   primary_title    47331 non-null  object 
 3   original_title   47331 non-null  object 
 4   start_year       47331 non-null  int64  
 5   end_year         47331 non-null  object 
 6   runtime_minutes  47331 non-null  object 
 7   is_adult         47331 non-null  int64  
 8   genres           47331 non-null  object 
 9   average_rating   47329 non-null  float64
 10  votes            47329 non-null  float64
 11  review           47331 non-null  object 
 12  rating           47331 non-null  int64  
 13  sp               47331 non-null  object 
 14  pos              47331 non-null  int64  
 15  ds_part          47331 non-null  object 
 16  idx              47331 non-null  int64  
dtypes: float64(2

In [4]:
df_review.head()

,tconst,title_type,primary_title,original_title,start_year,end_year,runtime_minutes,is_adult,genres,average_rating,votes,review,rating,sp,pos,ds_part,idx
0,tt0068152,movie,$,$,1971,\N,121,0,"Comedy,Crime,Drama",6.3,2218.0,The pakage implies that Warren Beatty and Gold...,1,neg,0,train,8335
1,tt0068152,movie,$,$,1971,\N,121,0,"Comedy,Crime,Drama",6.3,2218.0,How the hell did they get this made?! Presenti...,1,neg,0,train,8336
2,tt0313150,short,'15','15',2002,\N,25,0,"Comedy,Drama,Short",6.3,184.0,There is no real story the film seems more lik...,3,neg,0,test,2489
3,tt0313150,short,'15','15',2002,\N,25,0,"Comedy,Drama,Short",6.3,184.0,Um .... a serious film about troubled teens in...,7,pos,1,test,9280
4,tt0313150,short,'15','15',2002,\N,25,0,"Comedy,Drama,Short",6.3,184.0,I'm totally agree with GarryJohal from Singapo...,9,pos,1,test,9281


Despues de importar los datos, hemos visto que en general se encuentran correctamente, los nombres se encuentran normalizados, y salvo `average_rating` y `votes` que a cada uno tienen dos valores ausentes, caba destacar que en especial de todo este conjunto de datos, centraremos nuestra atención en 3 características para resolverlo, estas son: `review`, `pos`, `ds_part`. En estas tres características centraremos nuestro análisis. 

# Etapa 2. Preparación de los Datos <a id='data_preparing'></a>

In [5]:
# Debido a que solo nos centraremos en 3 características las demás las vamos a eliminar, para centrarnos solo en las 3 a utilizar.
df_review = df_review[['review', 'pos', 'ds_part']]

In [6]:
df_review.head()

,review,pos,ds_part
0,The pakage implies that Warren Beatty and Gold...,0,train
1,How the hell did they get this made?! Presenti...,0,train
2,There is no real story the film seems more lik...,0,test
3,Um .... a serious film about troubled teens in...,1,test
4,I'm totally agree with GarryJohal from Singapo...,1,test


In [7]:
# Después del head tenemos que ver a mas detalle la característica pos
df_review['pos'].value_counts()

pos
0    23715
1    23616
Name: count, dtype: int64

## Análisis del Desequilibrio de Clases

Las clases de reseñas positivas y negativas están **prácticamente balanceadas**. La diferencia entre el número de reseñas positivas y negativas es mínima, lo que significa que el conjunto de datos no presenta un **desequilibrio de clases significativo**. Esto es positivo, ya que los modelos de clasificación tienden a funcionar mejor cuando las clases están equilibradas, ya que no están sesgados hacia una clase en particular.

Este equilibrio entre las clases permite que los modelos entrenen y evalúen las reseñas sin una tendencia inherente a predecir una clase más que la otra. Sin embargo, en caso de que se presenten diferencias significativas en el futuro, se pueden explorar técnicas como el **sobremuestreo (oversampling)** de la clase minoritaria o el **submuestreo (undersampling)** de la clase mayoritaria para corregir el desequilibrio.


In [8]:
# Función que normaliza, elimina puntuación, stopwords y lematiza
def preprocess_review(text):
    # Convertir a minúsculas
    text = text.lower()
    
    # Eliminar signos de puntuación y números
    words = word_tokenize(text)
    words = [word for word in words if word.isalpha()]
    
    # Eliminar stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Lematizar las palabras
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    
    # Unir las palabras procesadas en una cadena
    return ' '.join(lemmatized_words)

# Aplicar la función a la columna 'review'
df_review['review'] = df_review['review'].apply(preprocess_review)


In [9]:
# Filtrar los datos de entrenamiento (ds_part == 'train') y de prueba (ds_part == 'test')
train_data = df_review[df_review['ds_part'] == 'train']
test_data = df_review[df_review['ds_part'] == 'test']

# Características (reseñas) y etiquetas (pos) para el entrenamiento
features_train = train_data['review']
target_train = train_data['pos']

# Características (reseñas) y etiquetas (pos) para la prueba
features_test = test_data['review']
target_test = test_data['pos']

# Inicializar el vectorizador TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)

# Ajustar y transformar las reseñas de entrenamiento
features_train_tfidf = vectorizer.fit_transform(features_train)

# Transformar las reseñas de prueba (sin hacer fit nuevamente)
features_test_tfidf = vectorizer.transform(features_test)

# Ver el tamaño de la matriz resultante para entrenamiento y prueba
print(f"Tamaño de features_train_tfidf: {features_train_tfidf.shape}")
print(f"Tamaño de features_test_tfidf: {features_test_tfidf.shape}")

Tamaño de features_train_tfidf: (23796, 5000)
Tamaño de features_test_tfidf: (23535, 5000)


# Etapa 3. Entrenamiento de modelos <a id='data_training'></a>

In [10]:
# Inicializamos el modelo de regresión logística
model = LogisticRegression(max_iter=1000)

# Entrenar el modelo con los datos de entrenamiento
model.fit(features_train_tfidf, target_train)

# Hacer predicciones sobre el conjunto de prueba
predicciones = model.predict(features_test_tfidf)

# Mostrar el F1-score específico
f1 = f1_score(target_test, predicciones)
print(f"F1-Score: {f1}")

F1-Score: 0.8776647802221182


In [11]:
# Inicializamos el modelo de Árbol de Decisión
dt_model = DecisionTreeClassifier(random_state=42)

# Entrenamos el modelo
dt_model.fit(features_train_tfidf, target_train)

# Hacer predicciones sobre el conjunto de prueba
predicciones_dt = dt_model.predict(features_test_tfidf)

# Evaluar el F1-score
f1_dt = f1_score(target_test, predicciones_dt)
print(f"F1-Score de Árbol de Decisión: {f1_dt}")

F1-Score de Árbol de Decisión: 0.7050266827336891


In [13]:
# Inicializamos el modelo de Bosque Aleatorio (Random Forest)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenamos el modelo
rf_model.fit(features_train_tfidf, target_train)

# Hacer predicciones sobre el conjunto de prueba
predicciones_rf = rf_model.predict(features_test_tfidf)

# Evaluar el F1-score
f1_rf = f1_score(target_test, predicciones_rf)
print(f"F1-Score de Random Forest: {f1_rf}")

F1-Score de Random Forest: 0.8381565777448975


In [15]:
# Inicializamos el modelo XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Entrenamos el modelo
xgb_model.fit(features_train_tfidf, target_train)

# Hacer predicciones sobre el conjunto de prueba
predicciones_xgb = xgb_model.predict(features_test_tfidf)

# Evaluar el F1-score
f1_xgb = f1_score(target_test, predicciones_xgb)
print(f"F1-Score de XGBoost: {f1_xgb}")

c:\Users\Albert\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:11:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


F1-Score de XGBoost: 0.8515533165407221


In [16]:
# Reseñas de prueba manuales
new_reviews = [
    "This movie was absolutely amazing! I loved it!",   # Reseña positiva
    "Worst movie ever, I couldn't sit through it.",    # Reseña negativa
    "An incredible performance by the lead actor!",    # Reseña positiva
    "A total waste of time, would never recommend it."  # Reseña negativa
]

In [17]:
# Preprocesar las reseñas y vectorizarlas
new_reviews_preprocessed = [preprocess_review(review) for review in new_reviews]
new_reviews_tfidf = vectorizer.transform(new_reviews_preprocessed)

In [19]:
# Clasificación con cada modelo
predictions_logreg = model.predict(new_reviews_tfidf)
predictions_rf = rf_model.predict(new_reviews_tfidf)
predictions_xgb = xgb_model.predict(new_reviews_tfidf)
predictions_dt = dt_model.predict(new_reviews_tfidf)

# Mostrar los resultados
print(f"Logistic Regression Predictions: {predictions_logreg}")
print(f"Random Forest Predictions: {predictions_rf}")
print(f"XGBoost Predictions: {predictions_xgb}")
print(f"Decision Tree Predictions: {predictions_dt}")

Logistic Regression Predictions: [1 0 1 0]
Random Forest Predictions: [1 0 1 0]
XGBoost Predictions: [1 0 1 0]
Decision Tree Predictions: [1 0 1 0]



### **Predicciones de los Modelos**:

Las siguientes son las predicciones realizadas por cada uno de los modelos para las 4 reseñas de prueba:

| Reseña                              | Regresión Logística | Random Forest | XGBoost | Árbol de Decisión |
|-------------------------------------|---------------------|---------------|---------|-------------------|
| **Reseña 1**: "This movie was absolutely amazing! I loved it!" | Positiva (1)        | Positiva (1)   | Positiva (1) | Positiva (1)      |
| **Reseña 2**: "Worst movie ever, I couldn't sit through it." | Negativa (0)        | Negativa (0)   | Negativa (0) | Negativa (0)      |
| **Reseña 3**: "An incredible performance by the lead actor!" | Positiva (1)        | Positiva (1)   | Positiva (1) | Positiva (1)      |
| **Reseña 4**: "A total waste of time, would never recommend it." | Negativa (0)        | Negativa (0)   | Negativa (0) | Negativa (0)      |

### **Explicación de los Resultados**:

1. **Reseña 1**: "This movie was absolutely amazing! I loved it!"
   - **Predicción de todos los modelos**: Positiva (1)
   - **Análisis**: Todos los modelos predicen correctamente que esta reseña es **positiva**. Esto es esperable, ya que las palabras "absolutely amazing" y "loved" son indicativos claros de una reseña positiva.

2. **Reseña 2**: "Worst movie ever, I couldn't sit through it."
   - **Predicción de todos los modelos**: Negativa (0)
   - **Análisis**: Todos los modelos predicen correctamente que esta reseña es **negativa**. Palabras como "Worst" y "couldn't sit through it" claramente indican una reseña negativa.

3. **Reseña 3**: "An incredible performance by the lead actor!"
   - **Predicción de todos los modelos**: Positiva (1)
   - **Análisis**: Nuevamente, todos los modelos predicen correctamente que esta reseña es **positiva**. El uso de la palabra "incredible" en conjunto con "performance" y "lead actor" sugiere una evaluación positiva de la película.

4. **Reseña 4**: "A total waste of time, would never recommend it."
   - **Predicción de todos los modelos**: Negativa (0)
   - **Análisis**: Todos los modelos predicen correctamente que esta reseña es **negativa**. Las palabras "total waste of time" y "would never recommend" son típicamente asociadas a una opinión negativa sobre una película.

### **Resumen de Resultados**:

- Todos los modelos, **Regresión Logística**, **Random Forest**, **XGBoost** y **Árbol de Decisión**, hicieron las **predicciones correctas** para todas las reseñas de prueba. Es importante destacar que este conjunto de reseñas es sencillo, con términos muy claros para clasificarlas como positivas o negativas.
  

Dado que todos los modelos dieron las mismas predicciones en este conjunto de reseñas específicas, podemos concluir que:

- **Todos los modelos parecen estar funcionando correctamente** para tareas de clasificación de texto cuando las reseñas son claras y contienen términos indicativos evidentes de ser positivas o negativas.
- Sin embargo, este conjunto de reseñas es relativamente sencillo. En escenarios más complejos, los modelos podrían diverger en sus predicciones, especialmente cuando las reseñas contienen ambigüedades o un lenguaje más sofisticado.



# Conclusiones finales del proyecto <a id='end'></a>

### **Descripción del Proyecto:**
En este proyecto, nuestro objetivo fue entrenar modelos de clasificación para predecir las reseñas de películas como positivas o negativas. Utilizamos el conjunto de datos **imdb_reviews.tsv** que contiene reseñas de películas de IMDB, donde cada reseña está etiquetada como **0** para negativa y **1** para positiva. Se nos pidió que entrenáramos al menos tres modelos diferentes y alcanzáramos un **F1-score** de al menos **0.85** en las predicciones.

### **Análisis de las Clases:**
Al analizar la distribución de las clases, observamos que las clases están **balanceadas**, con **23,715 reseñas negativas** y **23,616 reseñas positivas**. Este equilibrio facilita el entrenamiento de los modelos, ya que no existe un sesgo inherente hacia ninguna de las clases.

### **Modelos Entrenados:**
Se entrenaron cuatro modelos diferentes para este proyecto:

1. **Regresión Logística**
2. **Árbol de Decisión**
3. **Bosque Aleatorio (Random Forest)**
4. **XGBoost**

Cada uno de estos modelos fue entrenado utilizando los datos preprocesados, y se evaluó su rendimiento utilizando el **F1-score**.

### **Resultados de los Modelos:**
- **Regresión Logística**: F1-score de **0.88**. Este modelo mostró un buen rendimiento, clasificando las reseñas de manera efectiva entre positivas y negativas.
- **Árbol de Decisión**: F1-score de **0.70**. Aunque el rendimiento de este modelo fue inferior, aún logró una clasificación razonable, pero mostró signos de sobreajuste.
- **Bosque Aleatorio**: F1-score de **0.84**. Este modelo mostró un rendimiento sólido, con una capacidad de generalización adecuada para las reseñas.
- **XGBoost**: F1-score de **0.85**. Este modelo mostró el mejor rendimiento, alcanzando el umbral requerido de F1-score de **0.85**.

### **Clasificación de Nuevas Reseñas:**
Realizamos pruebas adicionales con reseñas de prueba manuales. Los modelos clasificaron las reseñas de manera consistente, aunque algunos modelos (como el Árbol de Decisión) mostraron algunas predicciones incorrectas en casos más complejos.

### **Diferencias entre los Modelos:**
Al comparar los modelos, **XGBoost** y **Bosque Aleatorio** destacaron por su rendimiento consistente y alto en el conjunto de prueba. **Árbol de Decisión**, por su parte, mostró un rendimiento inferior debido a su tendencia al sobreajuste, especialmente con datos que contienen palabras menos comunes o más ambiguas.

1. Los modelos basados en árboles, como **XGBoost** y **Random Forest**, fueron los más efectivos para este conjunto de datos.
2. Aunque el modelo de **Regresión Logística** mostró un rendimiento decente, los modelos más complejos, como **XGBoost**, superaron a los más simples.
3. El modelo **Árbol de Decisión** no fue tan efectivo debido a su capacidad limitada para generalizar cuando se enfrentó a datos más complejos.

Este proyecto demostró cómo entrenar y evaluar diferentes modelos de clasificación para predecir las reseñas de películas de IMDB. **XGBoost** y **Random Forest** mostraron un rendimiento superior y lograron superar el umbral del **F1-score de 0.85**, lo que confirma su efectividad para esta tarea. Las reseñas fueron correctamente clasificadas y se logró un análisis profundo sobre las diferencias de rendimiento entre los modelos.
